In [9]:
import pandas as pd
from collections import Counter
import requests
import json
import time

# Simulation d'une requête API REST qui se ferait à chaque ouverture de la page
Ici on fait un ensemble de requêtes auprès de l'API Jikan, pour récupérer depuis une liste d'ID les informations sur les animes qui nous intéressent pour les stocker par la suite

In [15]:


id_list = [21, 40748, 41433, 39555, 20787, 889, 269, 9919, 11633, 41468, 1, 35849, 1535, 28223, 38000, 35120, 31043, 5114, 384, 41353, 40052, 245, 42625, 11061, 36873, 40748, 14719, 18679, 31964, 10620, 32182, 39535, 37779, 20507, 29803, 22535, 30240, 13601, 40834, 40750, 34612, 205, 43690, 16498, 50265, 48453, 44942, 43697, 46569]
output_data = {}

for anime_id in id_list:
    url = f"https://api.jikan.moe/v4/anime/{anime_id}"
    response = requests.get(url)
    if response.status_code == 200:
        anime_data = json.loads(response.text)
        selected_data = {
            "images": anime_data["data"]["images"],
            "title": anime_data["data"]["title"],
            "status": anime_data["data"]["status"],
            "duration": anime_data["data"]["duration"],
            "synopsis": anime_data["data"]["synopsis"],
            "genres": [genre["name"] for genre in anime_data["data"]["genres"]],
            "note" : "&#9733 &#9733 &#9733 &#9733 &#9734", 
            "avis" :""
        }
        output_data[anime_id] = selected_data
        time.sleep(1.5)
    else:
        print(f"Failed to retrieve data for anime ID: {anime_id}")

with open("data.json", "w") as file:
    json.dump(output_data, file, indent=4)

print("Data saved to data.json")


Data saved to data.json


## Creation des Card en Html
Utilisation des donnée récoltés pour créer des cartes décrivant les animes choisis

In [8]:
with open("data.json", "r", encoding="utf-8") as file:
    data = json.load(file)

html_code = ""

for anime in data:
    title = anime["title"]
    status = anime["status"]
    duration = anime["duration"]
    synopsis = anime["synopsis"]
    genres = anime["genres"]
    genres_string = ", ".join(genres)
    genres_string2 = " ".join(genres)
    genres_string2 = genres_string2.lower()

    html_code += f'''
    <div class="col-lg-4 col-md-6 {genres_string2}">
        <div class="product-item">
            <div class="product-img">
                <img src="{anime['images']['jpg']['image_url']}" class="img-fluid d-block mx-auto">
            </div>
            <div class="product-content text-center py-3">
                <span class="d-block text-uppercase fw-bold" id="titre">{title}</span>
                <span class="d-block" id="status">{status} épisodes</span>
                <span class="d-block" id="synopsis">{synopsis}</span>
                <span class="d-block">Statut: {status}</span>
                <span class="d-block">Durée: {duration}</span>
                <span class="d-block">Genres: {genres_string}</span>
            </div>
        </div>
    </div>
    '''

# Read the index.html file
with open("index.html", "r", encoding="utf-8") as file:
    index_html = file.read()

# Find the placeholder <div id="anime-list"></div> in index.html and replace it with the generated HTML code
updated_index_html = index_html.replace('<div id="anime-list"></div>', f'<div id="anime-list" class="row justify-content-center g-4">{html_code}</div>')

# Write the updated HTML code back to index.html
with open("index.html", "w", encoding="utf-8") as file:
    file.write(updated_index_html)

print("Code HTML généré et enregistré dans index.html")


Code HTML généré et enregistré dans index.html


## Analyse des données pour choisir les filtres de genres
Analyse du nombre d'occurrences des genres et de la pertinence de ces derniers (choisir ceux qui décrivent le plus sans être redondant)

In [4]:
import json
import pandas as pd
from collections import Counter

# Charger les données depuis le fichier JSON
with open("data.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# Créer une liste avec tous les genres
all_genres = [genre for anime in data for genre in anime["genres"]]

# Créer un dictionnaire de genres avec leur nombre d'occurrences
genre_counts = dict(Counter(all_genres))

# Créer une matrice de corrélation des genres
correlation_matrix = pd.DataFrame(0, index=genre_counts.keys(), columns=genre_counts.keys())

# Parcourir chaque anime pour mettre à jour la matrice de corrélation
for anime in data:
    genres = anime["genres"]
    for genre1 in genres:
        for genre2 in genres:
            if genre1 != genre2:
                correlation_matrix.loc[genre1, genre2] += 1

# Afficher la matrice de corrélation
print("Tableau de corrélation des genres :")
print(correlation_matrix)


Tableau de corrélation des genres :
               Action  Adventure  Fantasy  Award Winning  Sci-Fi  Sports  \
Action              0          9       16              6       7       0   
Adventure           9          0        8              0       0       0   
Fantasy            16          8        0              3       1       0   
Award Winning       6          0        3              0       1       0   
Sci-Fi              7          0        1              1       0       0   
Sports              0          0        0              0       0       0   
Mystery             3          1        1              0       2       0   
Comedy              5          1        1              1       0       0   
Drama               6          2        4              1       2       0   
Romance             1          0        0              0       1       0   
Supernatural        4          1        0              0       0       0   
Suspense            3          0        1           

Les critères pour être représentatif : 
- Avoir un grand nombre d'occurrence
- Tous les anime doivent avoir au moins un des genres représentatifs
- Ne doit pas être toujours présent avec le même autre genre (pas assez discriminatif)

In [7]:
from collections import Counter

# Obtenir les genres avec les occurrences les plus élevées
top_genres = sorted(genre_counts.items(), key=lambda x: x[1], reverse=True)

# Initialiser avec les genres du premier élément
selected_genres = set(data[0]["genres"])

# Filtrer les genres en fonction des critères
for genre, count in top_genres:
    if len(selected_genres) == 6:
        break
    if all(correlation_matrix.loc[genre, selected_genre] <= 1 for selected_genre in selected_genres):
        selected_genres.add(genre)

# Trier les genres sélectionnés par nombre d'occurrences
selected_genres = sorted(selected_genres, key=lambda x: genre_counts[x], reverse=True)

# Afficher les 6 genres les plus représentatifs
print("Les 6 genres les plus représentatifs qui satisfont les critères spécifiés :")
for genre in selected_genres:
    print(genre)


Les 6 genres les plus représentatifs qui satisfont les critères spécifiés :
Action
Fantasy
Adventure
Ecchi
Romance
Sports
